In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class  DataPreprocessConfig:
    root_dir:Path
    input_preprocess:Path
    image_input:Path
    text_input:Path
    output_preprocess:Path
    unzip_dir:Path



In [4]:
from Image2Recipie.constants import *
from Image2Recipie.utils.common import read_yaml,create_directories

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bisht\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.preprocessing.root_dir])

    def get_data_preprocessing_config(self)->DataPreprocessConfig:
        config=self.config
        create_directories([config.preprocessing.root_dir])
        create_directories([config.preprocessing.input_preprocess])
        create_directories([config.preprocessing.output_preprocess])
        data_preprocess_config=DataPreprocessConfig(
            root_dir=config.preprocessing.root_dir,
            input_preprocess=config.preprocessing.input_preprocess,
            image_input=config.preprocessing.image_input,
            text_input=config.preprocessing.text_input,
            output_preprocess=config.preprocessing.output_preprocess,
            unzip_dir=config.data_ingestion.unzip_dir
        )
        return data_preprocess_config

In [6]:
import os
import urllib.request as request 
import gdown
from Image2Recipie import logger
import tensorflow as tf
from Image2Recipie.utils.common import save_object,get_maxlen,clean_ingredients,CustomTokenizer,detect_english_text
import cv2 as cv
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [10]:
class DataPreprocess:
    def __init__(self,config:DataPreprocessConfig):
        self.config = config
        self.input_image=None 
    
        

    def output_preprcess(self):
        self.data=pd.read_csv(os.path.join(self.config.unzip_dir,'cuisine_updated.csv'))
        self.features={}
        self.data['cleaned_ingredient']=self.data['cleaned_ingredient'].apply(clean_ingredients)
        self.data['cleaned_ingredient']=self.data['cleaned_ingredient'].apply(lambda x:','.join([x.strip() for x in x.split(',')]))
        tokenizer=CustomTokenizer()
        y_tokenized=tokenizer.fit_transform(self.data['cleaned_ingredient'])
       
        max_output=max(len(x) for x in y_tokenized)
        ytoken=pad_sequences(y_tokenized,maxlen=max_output,padding='post')
        y_output=to_categorical(ytoken,num_classes=None)
        y_encoder_input=[]
        for i in range(0,ytoken.shape[0]):
            y_encoder_input.append((np.insert(ytoken[i][:-1],0,0)))
        y_encoder=np.array(y_encoder_input)
        
        self.features['max_outputtoken']=max_output
        self.features['output_vocab_size']=len(tokenizer.get_word_index())
        save_object(os.path.join(self.config.input_preprocess,'y_input_encoder.pkl'),y_encoder)
        save_object(os.path.join(self.config.output_preprocess,'y_tokenized.pkl'),y_output)
        save_object(os.path.join(self.config.output_preprocess,'tokenizer.pkl'),tokenizer)
        
        logger.info(f'The Preprocessed output is saved in {self.config.output_preprocess}')
    def input_preprocess(self):
        non_english=detect_english_text(df=self.data)
        non_english=detect_english_text(df=self.data)
        self.data.drop(index=non_english,inplace=True)
        self.data.reset_index(drop=True,inplace=True)
        
        
        
        maxlen=get_maxlen(self.data['name'])
        token=Tokenizer()
        token.fit_on_texts(self.data['name'])
        
        Xtoken=token.texts_to_sequences(self.data['name'])
        Xtoken=pad_sequences(Xtoken,maxlen=maxlen)
        self.features['input_maxlen']=maxlen
        vocab_size=len(token.word_index)
        self.features['input_vocal_size']=vocab_size
        X=[]
        for i in self.data['image_path']:
            
            img = cv.imread(os.path.join(self.config.unzip_dir, "data/data/" + i))
            img=cv.resize(img,(160,160))
            X.append(img)
        input_image=np.array(X)
        save_object(self.config.image_input,input_image)
        save_object(self.config.text_input,Xtoken)
        save_object(os.path.join(self.config.input_preprocess,'input_tokenizer.pkl'),token)
        print(self.features)
        save_object(os.path.join(self.config.root_dir,'features.pkl'),self.features)
        logger.info(f'The Preprocessed input is saved in {self.config.input_preprocess}')




        
    

            
            





In [11]:
try:
    config=ConfigurationManager()
    data_preprocessing_config=config.get_data_preprocessing_config()
    data_preprocessing=DataPreprocess(config=data_preprocessing_config)
    
    data_preprocessing.output_preprcess()
    data_preprocessing.input_preprocess()
    
except Exception as e:
    raise e

[2024-11-03 22:58:52,188:INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-11-03 22:58:52,207:INFO:common:yaml file: params.yaml loaded successfully]
[2024-11-03 22:58:52,214:INFO:common:created directory at: artifacts/preprocessing]
[2024-11-03 22:58:52,222:INFO:common:created directory at: artifacts/preprocessing]
[2024-11-03 22:58:52,224:INFO:common:created directory at: artifacts/preprocessing/inputs_preprocess]
[2024-11-03 22:58:52,225:INFO:common:created directory at: artifacts/preprocessing/output_preprocess]


[2024-11-03 22:59:00,290:INFO:1171314784:The Preprocessed output is saved in artifacts/preprocessing/output_preprocess]
{'max_outputtoken': 44, 'output_vocab_size': 992, 'input_maxlen': 80, 'input_vocal_size': 2391}
[2024-11-03 22:59:52,914:INFO:1171314784:The Preprocessed input is saved in artifacts/preprocessing/inputs_preprocess]


In [9]:
df[df['cleaned_ingredient'].isnull()]

NameError: name 'df' is not defined

'\n'